# Introduction

This tutorial demonstrates different routes to obtain strain maps from scanning electron diffraction data.

The code functionality is illustrated using synthetic data, which is first generated using pyxem. This synthetic data represents a simple cubic crystal that is distorted to a tetragonal stucture. The intention is for this to provide an easy to understand illustration of the code functionality rather than to model any physical system.

All functionality has been checked to run using pyxem-0.9.0

# Contents

1. <a href='#gen'> Setting up & Creating Synthetic Data</a>
2. <a href='#aff'> Image Affine Transform Based Mapping</a>
3. <a href='#vec'> Vector Based Mapping</a>

# <a id='gen'></a> 1. Setting up & Creating Synthetic Data

Import pyxem and other required libraries

In [ ]:
%matplotlib tk
import pyxem as pxm
import numpy as np
import hyperspy.api as hs
import diffpy.structure
from matplotlib import pyplot as plt
from pyxem.generators.indexation_generator import IndexationGenerator
from pyxem.utils.sim_utils import sim_as_signal

Define a structure for the creation of synthetic data

In [ ]:
latt = diffpy.structure.lattice.Lattice(3,3,3,90,90,90)
atom = diffpy.structure.atom.Atom(atype='Ni',xyz=[0,0,0],lattice=latt)
structure = diffpy.structure.Structure(atoms=[atom],lattice=latt)

Simulate an electron diffraction pattern

In [ ]:
ediff = pxm.DiffractionGenerator(300., 0.025)
diffraction = ediff.calculate_ed_data(structure, reciprocal_radius=5.)
pattern = sim_as_signal(diffraction,128,0.05,1).data

Define a distorted structure and simulate diffraction

In [ ]:
latt = diffpy.structure.lattice.Lattice(3+0.12,3+0.12,3,90,90,90)
atom = diffpy.structure.atom.Atom(atype='Ni',xyz=[0,0,0],lattice=latt)
structure_d = diffpy.structure.Structure(atoms=[atom],lattice=latt)
diffractiond = ediff.calculate_ed_data(structure_d, reciprocal_radius=5.)
patternd = sim_as_signal(diffractiond,128,0.05,1).data

Copy the data and stitch patterns together with distortions applied to the patterns to make a 2x2 map

In [ ]:
dp = pxm.ElectronDiffraction2D((np.asarray([[pattern,patternd],[pattern,pattern]])))

x_l = []
for x in [0, 0, -0.01, 0.02]:
    x_s = np.eye(3)
    x_s[0,0] += x
    x_l.append(x_s)

angles = hs.signals.Signal2D(np.asarray(x_l).reshape(2,2,3,3))
dp = dp.apply_affine_transformation(D=angles,order=5,inplace=False)
dp.set_diffraction_calibration(1)

Plot the synthetic data to visualise distortions to be mapped

In [ ]:
dp.plot(cmap='inferno')

# <a id='aff'></a> 2. Image Affine Transform Based Mapping

Import the ScalableReferencePattern component that will be affine transformed to fit the data

In [ ]:
from pyxem.components.scalable_reference_pattern import ScalableReferencePattern

Create a model containing the reference pattern component

In [ ]:
m = dp.create_model()
ref = ScalableReferencePattern(dp.inav[0,0])
m.append(ref)

Print the affine transform values associated with the distorted diffraction pattern before fitting.

In [ ]:
m.print_current_values()

Perform fitting

In [ ]:
m.multifit()

Construct the displacement graident tensor at each pixel from the fitting results

In [ ]:
disp_grad = ref.construct_displacement_gradient()

Perform (right) polar decomposition on the displacement gradient tensor to get rotation matrix, R, and strain matrix, U, at each pixel.

In [ ]:
R, U = disp_grad.polar_decomposition()

Get a strain map and plot it.

In [ ]:
strain_map = disp_grad.get_strain_maps()
strain_map.plot(cmap='seismic',vmax=0.04,vmin=-0.04)

# <a id='vec'></a> 6. Vector Based Mapping

Import pyxem modules for vector based strain mapping

In [ ]:
from pyxem.generators.subpixelrefinement_generator import SubpixelrefinementGenerator
from pyxem.signals.tensor_field import *
from pyxem.generators.displacement_gradient_tensor_generator import *

Finding the two peaks to be used for strain mapping

In [ ]:
dp.plot()

x_peak = [24,0]
y_peak = [0,-42]

Determine peak positions to subpixel precision

In [ ]:
spg = SubpixelrefinementGenerator(dp,np.asarray([x_peak,y_peak]))
Vs = spg.center_of_mass_method(6)

Compare distorted and undistorted diffraction vectors to obtain a strain map

In [ ]:
D = get_DisplacementGradientMap(hs.signals.Signal2D(Vs), Vs.data[0,0])

In [ ]:
strain_map = D.get_strain_maps()

In [ ]:
strain_map.plot(cmap='seismic',vmax=0.04,vmin=-0.04)